In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [3]:
model = tf.keras.models.load_model('model.h5')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    ohc_geo = pickle.load(file)

2025-10-22 21:26:12.907395: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-22 21:26:12.911515: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
sample = pd.DataFrame([{
    'CreditScore': 600,
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Geography': 'France'
}])

In [5]:
# ========== Encode Gender ==========
sample['Gender'] = label_encoder_gender.transform(sample['Gender'])

# ========== Encode Geography ==========
geo_encoded = ohc_geo.transform(sample[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=ohc_geo.get_feature_names_out(['Geography']))

# Merge with the main sample
sample = pd.concat([sample.drop('Geography', axis=1), geo_encoded_df], axis=1)

# ========== Reorder columns to match training data ==========
# These must match the same order used in training (X_train.columns)
expected_columns = [
    'CreditScore', 'Gender', 'Age', 'Tenure', 'Balance',
    'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
    'Geography_France', 'Geography_Germany', 'Geography_Spain'
]

In [6]:
# Add any missing columns (e.g., if sample didn't have a country)
for col in expected_columns:
    if col not in sample.columns:
        sample[col] = 0

sample = sample[expected_columns]

# ========== Scale the data ==========
sample_scaled = scaler.transform(sample)

# ========== Predict ==========
prediction = model.predict(sample_scaled)
predicted_class = (prediction > 0.5).astype(int)

# ========== Output ==========
print(f"Predicted probability of churn: {prediction[0][0]:.4f}")
print(f"Predicted class: {'Will churn' if predicted_class[0][0] == 1 else 'Will NOT churn'}")


1/1 [==============================] - 0s 116ms/step
Predicted probability of churn: 0.0442
Predicted class: Will NOT churn
